In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
from urllib.parse import urljoin
import os
from concurrent.futures import ThreadPoolExecutor
import re
import csv



def clear_table():
    try:
        # Update the table name accordingly
        sql = "DELETE FROM books"
        mysql_cursor.execute(sql)
        mysql_conn.commit()
        print("Table cleared successfully")
    except Exception as e:
        print(f"Error clearing table: {e}")
        
def get_book_url():
    base_url = f"http://books.toscrape.com/catalogue/"
    # 遍历所有页面
    all_links = []
    for i in range(1,51):
        url = base_url + f'page-'+str(i)+f'.html'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = [urljoin(base_url, a['href']) for h3 in soup.find_all('h3') for a in h3.find_all('a', href=True)]
        all_links.extend(links)
    return all_links

def scrape_book_info(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.find('h1').text.strip()
            image_url = urljoin(url, soup.find('div', class_='item active').find('img')['src'])
            price = soup.find('p', class_='price_color').text.strip()
            rating = soup.find('p', class_='star-rating')['class'][1]
            number = soup.find('h3').find('a')['href'].split('_')[-1].replace("/index.html", "")
            print([number, title, image_url, rating, price])
            download_image(image_url, folder, title)
            save_to_mysql(number, title, image_url, rating, price)
            save_to_csv([number, title, image_url, rating, price])

    except Exception as e:
        print(f"Error scraping {url}: {e}")

def download_image(image_url, folder, filename):
    # 使用正则表达式去除文件名中的非法字符并截断为较短的长度
    image_name = re.sub(r'[\/:*?"<>|]', '_', filename)[:100] + ".jpg"
    os.makedirs(folder, exist_ok=True)
    response = requests.get(image_url)
    with open(os.path.join(folder, f"{image_name}"), 'wb') as f:
        f.write(response.content)

def save_to_mysql(book_number, title, image_url, rating, price):
    try:
        sql = "INSERT INTO books (number, title, picture, rating, price) VALUES (%s, %s, %s, %s, %s)"
        values = (book_number, title, image_url, rating, price)
        mysql_conn.ping(reconnect=True)
        mysql_cursor.execute(sql, values)
        mysql_conn.commit()
        print("Book information saved to the database")
    except Exception as e:
        print(f"Error saving to the database: {e}")
        
def save_to_csv(data):
    header = ["Number", "Title", "Image URL", "Rating", "Price"]
    csv_filename = 'books.csv'
    if not os.path.exists(csv_filename):
        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(header)

    with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(data)


if __name__ == "__main__":
    # 初始化数据库连接
    mysql_conn = pymysql.connect(host='localhost', user='root', password='mysqlpassword', database='books_db')
    mysql_cursor = mysql_conn.cursor()

    folder = "./task3_picture"
    # 由于过程重复爬取，故每次爬取时先清除表的内容
    clear_table()
    urls = get_book_url()
    num_threads = 10
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        executor.map(scrape_book_info, urls)
    # 关闭数据库连接
    mysql_cursor.close()
    mysql_conn.close()


d:\Applications\Development\Anaconda3\envs\py36\lib\site-packages\pymysql\_auth.py:8: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.hazmat.backends import default_backend


Table cleared successfully
Error scraping http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html: 'NoneType' object has no attribute 'find'
['1000', 'Tipping the Velvet', 'http://books.toscrape.com/media/cache/08/e9/08e94f3731d7d6b760dfbfbc02ca5c62.jpg', 'One', 'Â£53.74']
['999', 'Soumission', 'http://books.toscrape.com/media/cache/ee/cf/eecfe998905e455df12064dba399c075.jpg', 'One', 'Â£50.10']
['997', 'Sapiens: A Brief History of Humankind', 'http://books.toscrape.com/media/cache/ce/5f/ce5f052c65cc963cf4422be096e915c9.jpg', 'Five', 'Â£54.23']
['995', 'The Dirty Little Secrets of Getting Your Dream Job', 'http://books.toscrape.com/media/cache/e1/1b/e11bea016d0ae1d7e2dd46fb3cb870b7.jpg', 'Four', 'Â£33.34']
['992', 'The Black Maria', 'http://books.toscrape.com/media/cache/d1/7a/d17a3e313e52e1be5651719e4fba1d16.jpg', 'One', 'Â£52.15']
['993', 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics', 'http://books.toscrape.com/media/

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib3.exceptions import InsecureRequestWarning
from bs4 import BeautifulSoup
import pymysql
from urllib.parse import urljoin
import os
from concurrent.futures import ThreadPoolExecutor
import csv
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import random

def get_ip():
    # 快代理的ip
    url = "http://api.tianqiip.com/getip?secret=cx1xlfx52o8s1rdq&num=5&type=txt&lb=%5Cn&region=440000&port=1&time=5&mr=1&sign=15fb650d0f872b891e37db13014c89d7"
    res = requests.get(url)
    print(res.text)
    if res.status_code==200:
        prox = res.text
    prox = res.text.split("\n")[:-1]
    proxy = [{'http':'http://'+i} for i in prox]
    # 使用前可判断一下是否可用，也可能返回其他state code，具体就依据具体情况解决
    print(proxy)
    ts = "http://www.baidu.com/"
    # ip = []
    for j in proxy:
        try:
            r = requests.get(ts,proxies=j)
        except:
            print("ip不可用")
    return prox

# 使用ip
# def download_pages(url):
#     print(url)
#     ip_list = get_ip()
#     headers = {
#             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
#             'cookie': 'ZFY=5S:B8OLUq:BOD8mVNCD4o0ZmLhkOrHTzZooj6LRBWCa2E:C; BCLID_BFESS=12915262214570578280; BDSFRCVID_BFESS=CQ-OJeC62GDmcJvfz9aVEHtYogbM-ARTH6_nOMpJ2ushD7GJP9EHEG0PpU8g0KAMOXYnogKK5mOTH65FCeIKEZjgOUBLV6qSzhY6tf8g0f5; H_BDCLCKID_SF_BFESS=Jn-8oC0KtDK3fP36q4Job-_ObhuX5-RLf5Tv_p7F5l8-hR4mbfbY545LKnn9qURu35r7QJ45WIQxOKQ3hxRjQxATBPcJQ6cQJeOKWfTN3KJmSbL9bT3vjMrBhR5u2-biWbTL2MbdJqvP_IoG2Mn8M4bb3qOpBtQmJeTxoUJ25DnJhbLGe6KKj6Q-eHueqbTeM6-X3JjV5PK_Hn7zep5K5M4pbtbC56b7KDCJ2bLhbKJ4f4OaytDKyUnQbPnnBT572nTkXCn7tfQ_D4D93RQPDTDkQN3Tb4KO5bRiLRolyPOWDn3oyToVXp0nj4Rly5jtMgOBBJ0yQ4b4OR5JjxonDh83bG7MJUutfJAJoCIKtC02bP365ITJhPAthxR22D62aKDs5MDEBhcqEIL4QUOV-UIBblC85l54WTnkWb3tB66NVxbSjln_XUuQyG5tb-vRJ2vZbITNWp5nhMJx3j7JDMPFXtnvX46y523i_R6vQpP-MqQJbfrvQqIqbPnP3f7pJj6gXJnwJtbtbb0xb6_0-nDSHHDHtjDH3D; H_PS_PSSID=26350; BAIDUID_BFESS=63794D23AE5CA338662673806B2AE198:FG=1; BDUSS_BFESS=WNiOXNZRlAybHNWVzRJcExZeVh3cVFBUzczaFZFOHhRRXJwaDFmZDB5R2ptUzlsSVFBQUFBJCQAAAAAAQAAAAEAAAA9UMk-sK67qrbByukAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAKMMCGWjDAhle; ab_sr=1.0.1_NDFmOGUwZDRjMDg3YTU4NDY0NTgzMzVhYmM4ZmExMjc4NGFmZTM5YWU4OTVkZjUwYWQ4M2Y0YjdmNDdjYjcxNjYwMDdkNTE2ZDBmMzIwNWU3MzY2OGY3NmI1MmQ5NTU2ZGFiYzg5NDVjNzQzZTdmNjY0NTUwZjk1Mjg0Y2UwNmVkYmUwMmQxZDQ0ZGMxMzM1ZGZiZWRiYzk0MWUxNjRkMTBlNWQxM2QxNTA0M2ViMmQ4ZWFjNjA0N2YzMzJhMDBi',
#             'Connection':'close'}
#     ip = random.choice(ip_list)
#     ip = '125.89.44.123:40018'
#     proxies = {"http":f"http://{ip}"}
#     try:
#         response = requests.get(url, headers=headers, stream=True, verify=False, proxies=proxies, timeout=2)
#         response.raise_for_status()  # Raise an HTTPError for bad responses
#         return response.content.decode('utf-8')  # Decode the bytes to a string
#     except requests.RequestException as e:
#         print(f"Error downloading page {url}: {e}")
#         return None
    
# 不使用ip
def download_pages(url):
    headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
            'cookie': 'ZFY=5S:B8OLUq:BOD8mVNCD4o0ZmLhkOrHTzZooj6LRBWCa2E:C; BCLID_BFESS=12915262214570578280; BDSFRCVID_BFESS=CQ-OJeC62GDmcJvfz9aVEHtYogbM-ARTH6_nOMpJ2ushD7GJP9EHEG0PpU8g0KAMOXYnogKK5mOTH65FCeIKEZjgOUBLV6qSzhY6tf8g0f5; H_BDCLCKID_SF_BFESS=Jn-8oC0KtDK3fP36q4Job-_ObhuX5-RLf5Tv_p7F5l8-hR4mbfbY545LKnn9qURu35r7QJ45WIQxOKQ3hxRjQxATBPcJQ6cQJeOKWfTN3KJmSbL9bT3vjMrBhR5u2-biWbTL2MbdJqvP_IoG2Mn8M4bb3qOpBtQmJeTxoUJ25DnJhbLGe6KKj6Q-eHueqbTeM6-X3JjV5PK_Hn7zep5K5M4pbtbC56b7KDCJ2bLhbKJ4f4OaytDKyUnQbPnnBT572nTkXCn7tfQ_D4D93RQPDTDkQN3Tb4KO5bRiLRolyPOWDn3oyToVXp0nj4Rly5jtMgOBBJ0yQ4b4OR5JjxonDh83bG7MJUutfJAJoCIKtC02bP365ITJhPAthxR22D62aKDs5MDEBhcqEIL4QUOV-UIBblC85l54WTnkWb3tB66NVxbSjln_XUuQyG5tb-vRJ2vZbITNWp5nhMJx3j7JDMPFXtnvX46y523i_R6vQpP-MqQJbfrvQqIqbPnP3f7pJj6gXJnwJtbtbb0xb6_0-nDSHHDHtjDH3D; H_PS_PSSID=26350; BAIDUID_BFESS=63794D23AE5CA338662673806B2AE198:FG=1; BDUSS_BFESS=WNiOXNZRlAybHNWVzRJcExZeVh3cVFBUzczaFZFOHhRRXJwaDFmZDB5R2ptUzlsSVFBQUFBJCQAAAAAAQAAAAEAAAA9UMk-sK67qrbByukAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAKMMCGWjDAhle; ab_sr=1.0.1_NDFmOGUwZDRjMDg3YTU4NDY0NTgzMzVhYmM4ZmExMjc4NGFmZTM5YWU4OTVkZjUwYWQ4M2Y0YjdmNDdjYjcxNjYwMDdkNTE2ZDBmMzIwNWU3MzY2OGY3NmI1MmQ5NTU2ZGFiYzg5NDVjNzQzZTdmNjY0NTUwZjk1Mjg0Y2UwNmVkYmUwMmQxZDQ0ZGMxMzM1ZGZiZWRiYzk0MWUxNjRkMTBlNWQxM2QxNTA0M2ViMmQ4ZWFjNjA0N2YzMzJhMDBi',
            'Connection':'keep-alive'}
    try:
        response = requests.get(url, headers=headers, stream=True, verify=False)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        return response.content.decode('utf-8')  # Decode the bytes to a string
    except requests.RequestException as e:
        print(f"Error downloading page {url}: {e}")
        return None
    
def get_doulist_movie_links(doulist_url, num_links=10):
    response = download_pages(doulist_url)
    # print(response)
    soup = BeautifulSoup(response, 'html.parser')
    
    # Extract href from class="post" elements
    post_elements = soup.find_all('div', class_='post')
    movie_links = [element.find('a')['href'] for element in post_elements]
    
    return movie_links[:num_links]

def get_reviews_links(movie_url):
    reviews_url = f"{movie_url}reviews?start="
    reviews_links = []

    start = 0
    while True:
        response = download_pages(f"{reviews_url}{start}")
        # print(f"{reviews_url}{start}")
        soup = BeautifulSoup(response, 'html.parser')
        
        # Extract href from review links
        review_elements = soup.find_all('h2')
        current_review_links = [element.find('a')['href'] for element in review_elements]

        # If no more reviews, break the loop
        if not current_review_links:
            break

        reviews_links.extend(current_review_links)
        start += 20  # Assuming 20 reviews per page, adjust if needed

    return reviews_links

def save_to_mysql(movie, author, review):
    try:
        sql = "INSERT INTO movies (movie, author, review) VALUES (%s, %s, %s)"
        values = (movie, author, review)
        mysql_conn.ping(reconnect=True)
        mysql_cursor.execute(sql, values)
        mysql_conn.commit()
        print("Movie information saved to the database")
    except Exception as e:
        print(f"Error saving to the database: {e}")

def save_to_csv(data):
    header = ["movie", "author", "review"]
    csv_filename = 'movies.csv'
    if not os.path.exists(csv_filename):
        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(header)

    with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(data)
        
def scrape_movie_info(url):
    try:
        response = download_pages(url)
        soup = BeautifulSoup(response, 'html.parser')
        review = soup.find('div', class_='review-content').get_text(separator='\n', strip=True)
        author = soup.find('header').find('a').find('span').get_text(strip=True)
        movie = soup.find('header').find_all('a')[1].get_text(strip=True)
        save_to_mysql(movie, author, review)
        save_to_csv([movie, author, review])
        # print(author,movie,review)

    except Exception as e:
        print(f"Error scraping {url}: {e}")

def get_review_urls(doulist_url):
    urls = []
    movie_links = get_doulist_movie_links(doulist_url, num_links=10)
    for movie_link in movie_links:
        reviews_links = get_reviews_links(movie_link)
        urls.extend(reviews_links)
    return urls
    

if __name__ == "__main__":
    doulist_url = "https://www.douban.com/doulist/5257286/"
    # 初始化数据库连接
    mysql_conn = pymysql.connect(host='localhost', user='root', password='mysqlpassword', database='books_db')
    mysql_cursor = mysql_conn.cursor()
    # 获取全部的影评链接
    urls = get_review_urls(doulist_url)
    # print(urls)
    # 多进程
    num_threads = 6
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        executor.map(scrape_movie_info, urls)
    # 关闭数据库连接
    mysql_cursor.close()
    mysql_conn.close()
    


Error saving to the database: 'NoneType' object has no attribute 'settimeout'
Movie information saved to the database
Movie information saved to the database
Movie information saved to the databaseError saving to the database: Packet sequence number wrong - got 1 expected 2

Movie information saved to the database
Movie information saved to the database
Error saving to the database: 'NoneType' object has no attribute 'read'Error saving to the database: Packet sequence number wrong - got 35 expected 2

Movie information saved to the database
Movie information saved to the database
Movie information saved to the database
Movie information saved to the database
Movie information saved to the database
Movie information saved to the database
Movie information saved to the database
Error saving to the database: (1366, "Incorrect string value: '\\xF0\\x9F\\x92\\x9D\\x0A\\xE5...' for column 'review' at row 1")
Movie information saved to the database
Movie information saved to the database
Movi